In [ ]:
import ee
import numpy as np
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# パッケージのインストール&インポート
!pip install rasterio
import numpy as np
import matplotlib.pyplot as plt
import rasterio

import json
import os
import glob

import time
from datetime import datetime
from dateutil.parser import parse

In [ ]:
!pip install folium

import folium

関心域のポリゴン作成およびその位置情報を取得するWebアプリを作成しました．以下のサイトより，関心粋の選択（左の作成チールより四角を選択），ポリゴンを作成。その後”Show feature”をクリックしポリゴン情報を右枠に表示させ、Copyを押して情報をコピーする。

In [ ]:
#関心領域のポリゴン情報の取得．
from IPython.display import HTML
HTML(r'<iframe width="1000" height="580" src="https://gispolygon.herokuapp.com/" frameborder="0"></iframe>')

ポリゴン情報をコピーし，以下のAの右辺にペーストする．

In [ ]:
A = {"type":"FeatureCollection","features":[{"properties":{"note":"","distance":"1127.16 m","drawtype":"rectangle","area":"11.71 ha"},"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[139.87487004138532,35.63225823758155],[139.87487004138532,35.6360949926148],[139.87790093757215,35.6360949926148],[139.87790093757215,35.63225823758155],[139.87487004138532,35.63225823758155]]]}}]}

In [ ]:
#今後使用する任意のファイル名をセットする． 例えば，地域の名前など．
object_name = 'Tokyo_TDL2'

In [ ]:
with open(str(object_name) +'_2.geojson', 'w') as f:
    json.dump(A, f)

In [ ]:
json_file = open(str(object_name) +'_2.geojson')
json_object = json.load(json_file)

In [ ]:
#jsonから関心域の緯度・経度情報のみを抽出する．

AREA = json_object["features"][0]["geometry"]['coordinates'][0]

In [ ]:
import pandas as pd

area = pd.DataFrame(AREA,
                  columns=['longtitude', 'latitude'])

In [ ]:
area_d =[[area['longtitude'].min(), area['latitude'].max()],
 [area['longtitude'].max(), area['latitude'].max()],
 [area['longtitude'].max(), area['latitude'].min()],
 [area['longtitude'].min(), area['latitude'].min()],
 [area['longtitude'].min(), area['latitude'].max()]]

In [ ]:
AREA = area_d
AREA

関心範囲の確認

In [ ]:
m = folium.Map([(AREA[0][1]+AREA[len(AREA)-2][1])/2,(AREA[0][0]+AREA[len(AREA)-3][0])/2], zoom_start=15)

folium.GeoJson(str(object_name) +'_2.geojson').add_to(m)
m

# Sentinel-1,2の観測画像の取得と解析

In [ ]:
region=ee.Geometry.Rectangle(area['longtitude'].min(),area['latitude'].min(), area['longtitude'].max(), area['latitude'].max())

In [ ]:
region['coordinates'][0]

Sentinel-1

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD

Dataset Availability
2014-10-03T00:00:00 - 2020-08-13T00:00:00



Sentinel-2 MSI: MultiSpectral Instrument, Level-1C 画像　

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2


2015-06-23T00:00:00 - 2020-08-13T00:00:00

SPACECRAFT_NAME	STRING	
Sentinel-2 spacecraft name: Sentinel-2A, Sentinel-2B

CLOUDY_PIXEL_PERCENTAGE	DOUBLE	
Granule-specific cloudy pixel percentage taken from the original metadata


In [ ]:
# 期間を指定
from_date='2019-01-01'
to_date='2020-08-31'

# 保存するフォルダ名
dir_name_s1 = 'GEE_Sentinel1_' + object_name
dir_name_s2 = 'GEE_Sentinel2_' + object_name

In [ ]:
#region=ee.Geometry.Rectangle([139.7123, 35.676, 139.7164, 35.680])

def cloudMasking(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10  
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

def ImageExport(image,description,folder,region,scale):
    task = ee.batch.Export.image.toDrive(image=image,description=description,folder=folder,region=region,scale=scale)
    task.start()

Sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(region).filterDate(parse(from_date),parse(to_date)).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).select(['VV'])
Sentinel2 = ee.ImageCollection('COPERNICUS/S2').filterBounds(region).filterDate(parse(from_date),parse(to_date)).filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 20).map(cloudMasking).select(['B4','B3','B2'])

imageList_s1 = Sentinel1.toList(300)
imageList_s2 = Sentinel2.toList(300) 

In [ ]:
for i in range(imageList_s1.size().getInfo()):
    image = ee.Image(imageList_s1.get(i))
    ImageExport(image.reproject(crs='EPSG:4326',scale=10),image.get('system:index').getInfo(),dir_name_s1,region['coordinates'][0],10)

In [ ]:
for i in range(imageList_s2.size().getInfo()):
    image = ee.Image(imageList_s2.get(i))
    ImageExport(image.reproject(crs='EPSG:4326',scale=10),image.get('system:index').getInfo(),dir_name_s2,region['coordinates'][0],10)

In [ ]:
# 時系列で可視化
s2_path = '/content/drive/My Drive/' + dir_name_s2 + '/'
files =os.listdir(s2_path)
files.sort()

plt.figure(figsize=(25, 25))
j=0

v = len(files)//5 +1 
for i in range(len(files)):
  # 画像を1シーンずつ取得して可視化
  with rasterio.open(s2_path + files[i]) as src:
      arr = src.read()
  j+=1# 画像のプロット位置をシフトさせ配置
  plt.subplot(v,5,j)
  arrayImg = np.asarray(arr).transpose(1,2,0).astype(np.float32)*2 #輝度を2倍に明るさを補正．
  plt.imshow(arrayImg)
  plt.title(files[i][0:8])# ファイル名から日付を取得
  #plt.tight_layout()

In [ ]:
s2_path = '/content/drive/My Drive/' + dir_name_s2 + '/'
files =os.listdir(s2_path)
files.sort()

n = 0
# データの読み込み
with rasterio.open(s2_path  + files[n]) as src:
    arr = src.read()

#chanelとh，vの位置を変える
arrayImg = np.asarray(arr).transpose(1,2,0).astype(np.float32)*2

print(files[n][0:8])
# 可視化
plt.imshow(arrayImg)

In [ ]:
n = len(files) - 1
# データの読み込み
with rasterio.open(s2_path  + files[n]) as src:
    arr = src.read()

#chanelとh，vの位置を変える
arrayImg = np.asarray(arr).transpose(1,2,0).astype(np.float32)*2

print(files[n][0:8])
# 可視化
plt.imshow(arrayImg)

In [ ]:
# 時系列で可視化
s1_path = '/content/drive/My Drive/' + dir_name_s1 + '/'
files =os.listdir(s1_path)
files.sort()

plt.figure(figsize=(20, 40))
j=0

v = len(files)//5 +1 
for i in range(len(files)):
  # 画像を1シーンずつ取得して可視化
  with rasterio.open(s1_path + files[i]) as src:
      arr = src.read()
  j+=1# 画像のプロット位置をシフトさせ配置
  plt.subplot(v,5,j)
  plt.imshow(arr[0], cmap='gray')
  plt.title(files[i][33:41])# ファイル名から日付を取得
  plt.tight_layout()

In [ ]:
# データの読み込み
n = 4

with rasterio.open(s1_path + files[n]) as src:
    arr = src.read()

print(files[n][33:41])
# 可視化
plt.imshow(arr[0], cmap='gray')

In [ ]:
# データの読み込み
n = len(files) -1

with rasterio.open(s1_path + files[n]) as src:
    arr = src.read()

print(files[n][33:41])
# 可視化
plt.imshow(arr[0], cmap='gray')

In [ ]:
# 当該エリアの反射強度の合計値を時系列グラフ化
sum_signal = []
label_signal = []
for i in range(len(files)):
  # 画像を1シーンずつ取得して可視化
  with rasterio.open(s1_path + files[i]) as src:
      arr = src.read()
  sum_signal.append(arr.sum())
  label_signal.append(files[i][33:41])

# 可視化
fig,ax = plt.subplots(figsize=(15,6))
plt.plot(sum_signal, marker='o')
ax.set_xticks(np.arange(0,len(files)))
ax.set_xticklabels(label_signal, rotation=90)
plt.title('Trend in parking lot usage at TDL.')
plt.xlabel('date')
plt.show()